## Explore YOLO output + triangulation

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()

In [2]:
detections = pd.read_csv(f"{root}/data/stereo_B_detections.csv")

In [3]:
detections.head()

,Unnamed: 0,frame_no,B1_x,B1_y,B1_confidence,B1_method,B2_x,B2_y,B2_confidence,B2_method
0,0,1061,16,374,0.797673,YOLO,1892,383,0.400000,LK
1,1,1062,71,383,0.832680,YOLO,1892,383,0.300000,LK
2,2,1063,60,382,0.400000,LK,1745,404,0.848748,YOLO
3,3,1064,178,402,0.845032,YOLO,1700,411,0.832230,YOLO
4,4,1065,225,410,0.840000,YOLO,1657,417,0.833924,YOLO


In [4]:
detection = detections.iloc[1]
detection.to_dict()

{'Unnamed: 0': 1,
 'frame_no': 1062,
 'B1_x': 71,
 'B1_y': 383,
 'B1_confidence': 0.8326795697212219,
 'B1_method': 'YOLO',
 'B2_x': 1892,
 'B2_y': 383,
 'B2_confidence': 0.3,
 'B2_method': 'LK'}

In [5]:

def load_stereo_a_maps():
    file_path = f"{root}/output/STEREO_A_rectification_params.xml"
    cv_file = cv2.FileStorage(file_path, cv2.FILE_STORAGE_READ)

    stereo_maps = {
        "CAM_1_map_x": cv_file.getNode("CAM_1_map_x").mat(),
        "CAM_1_map_y": cv_file.getNode("CAM_1_map_y").mat(),
        "CAM_2_map_x": cv_file.getNode("CAM_2_map_x").mat(),
        "CAM_2_map_y": cv_file.getNode("CAM_2_map_y").mat(),
        "CAM_1_projection_matrix": cv_file.getNode("CAM_1_projection_matrix").mat(),
        "CAM_2_projection_matrix": cv_file.getNode("CAM_2_projection_matrix").mat(),
        "disparity_to_depth_matrix": cv_file.getNode("disparity_to_depth_matrix").mat(),
        "CAM_1_roi": cv_file.getNode("CAM_1_roi").mat(),
        "CAM_2_roi": cv_file.getNode("CAM_2_roi").mat(),
    }

    cv_file.release()
    return stereo_maps

def load_stereo_b_maps():
    file_path = f"{root}/output/STEREO_B_rectification_params_2.xml"
    cv_file = cv2.FileStorage(file_path, cv2.FILE_STORAGE_READ)

    stereo_maps = {
        "CAM_3_map_x": cv_file.getNode("CAM_3_map_x").mat(),
        "CAM_3_map_y": cv_file.getNode("CAM_3_map_y").mat(),
        "CAM_4_map_x": cv_file.getNode("CAM_4_map_x").mat(),
        "CAM_4_map_y": cv_file.getNode("CAM_4_map_y").mat(),
        "CAM_3_projection_matrix": cv_file.getNode("CAM_3_projection_matrix").mat(),
        "CAM_4_projection_matrix": cv_file.getNode("CAM_4_projection_matrix").mat(),
        "disparity_to_depth_matrix": cv_file.getNode("disparity_to_depth_matrix").mat(),
        "CAM_3_roi": cv_file.getNode("CAM_3_roi").mat(),
        "CAM_4_roi": cv_file.getNode("CAM_4_roi").mat(),
    }

    cv_file.release()
    return stereo_maps

In [6]:
# load intrinsics and extrinsics

with open(f"{root}/output/intrinsic_params.json", "r") as f:
    intrinsic_params = json.load(f)
    
with open(f"{root}/output/1_stereo_params.json", "r") as f:
    stereo_params = json.load(f)
    
# necessary parameters
K_cam1_undistort = np.array(intrinsic_params['CAM_1']['K_undistort'])
D_cam1_undistort = np.array(intrinsic_params['CAM_1']['D'])

K_cam2_undistort = np.array(intrinsic_params['CAM_2']['K_undistort'])
D_cam2_undistort = np.array(intrinsic_params['CAM_2']['D'])

K_cam3_undistort = np.array(intrinsic_params['CAM_3']['K_undistort'])
D_cam3_undistort = np.array(intrinsic_params['CAM_3']['D'])

K_cam4_undistort = np.array(intrinsic_params['CAM_4']['K_undistort'])
D_cam4_undistort = np.array(intrinsic_params['CAM_4']['D'])


stereo_maps = load_stereo_a_maps()
x_map_cam1 = stereo_maps['CAM_1_map_x']
y_map_cam1 = stereo_maps['CAM_1_map_y']

print(f"Map shapes: {x_map_cam1.shape}, {y_map_cam1.shape}")

x_map_cam2 = stereo_maps['CAM_2_map_x']
y_map_cam2 = stereo_maps['CAM_2_map_y']

print(f"Map shapes: {x_map_cam2.shape}, {y_map_cam2.shape}")

stereo_maps_b = load_stereo_b_maps()
x_map_cam3 = stereo_maps_b['CAM_3_map_x']
y_map_cam3 = stereo_maps_b['CAM_3_map_y']

print(f"Map shapes: {x_map_cam3.shape}, {y_map_cam3.shape}")

x_map_cam4 = stereo_maps_b['CAM_4_map_x']
y_map_cam4 = stereo_maps_b['CAM_4_map_y']

print(f"Map shapes: {x_map_cam4.shape}, {y_map_cam4.shape}")



Map shapes: (2160, 3840), (2160, 3840)
Map shapes: (2160, 3840), (2160, 3840)
Map shapes: (2160, 3840), (2160, 3840)
Map shapes: (2160, 3840), (2160, 3840)


In [7]:
detection = detection.to_dict()
detection

{'Unnamed: 0': 1,
 'frame_no': 1062,
 'B1_x': 71,
 'B1_y': 383,
 'B1_confidence': 0.8326795697212219,
 'B1_method': 'YOLO',
 'B2_x': 1892,
 'B2_y': 383,
 'B2_confidence': 0.3,
 'B2_method': 'LK'}

In [8]:
# TODO: get 2D coordinates from YOLO output
# NOTE: measure = pixels
scale_factor = 2 
cam3_2d = (detection['B1_x'] * scale_factor, detection['B1_y'] * scale_factor)
cam4_2d = (detection['B2_x'] * scale_factor, detection['B2_y'] * scale_factor)

# TODO: undirtort the points
# P to normalize the points into pixel coordinates
undistorted_point_camera_3 = cv2.undistortPoints(cam3_2d, K_cam3_undistort, D_cam3_undistort, P=K_cam3_undistort)
undistorted_point_camera_4 = cv2.undistortPoints(cam4_2d, K_cam4_undistort, D_cam4_undistort, P=K_cam4_undistort)
print(f"cam 3 undistorted: {undistorted_point_camera_3}")
print(f"cam 4 undistorted: {undistorted_point_camera_4}")

# TODO: projection matrix
P1 = np.dot(K_cam3_undistort, np.hstack((np.eye(3), np.zeros((3, 1)))))
R2 = np.array(stereo_params['STEREO_B']['rotation_matrix']) 
T2 = np.array(stereo_params['STEREO_B']['translation_vector']) 
P2 = np.dot(K_cam4_undistort, np.hstack((R2, T2)))  # Camera 2's projection matrix

# TODO: triangulate
points_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistorted_point_camera_3, undistorted_point_camera_4)

# TODO: convert to 3D points
points_3d = points_3d_homogeneous[:3] / points_3d_homogeneous[3]
print(f"points 3d: {points_3d}")

cam 3 undistorted: [[[142. 766.]]]
cam 4 undistorted: [[[3767.1263821   769.89654481]]]
points 3d: [[-5253.06075604]
 [ -570.80681729]
 [ 5035.84424053]]


In [12]:
threshold = 15  # threshold in pixels

# Reproject 3D points (in mm) back to 2D
reprojected_point_1 = np.dot(P1, np.vstack((points_3d, np.ones((1, points_3d.shape[1])))))
reprojected_point_2 = np.dot(P2, np.vstack((points_3d, np.ones((1, points_3d.shape[1])))))

reprojected_point_1 /= reprojected_point_1[2]
reprojected_point_2 /= reprojected_point_2[2]

# NOTE: ravel() method is used to convert the 2D array into a 1D array
print(f"reprojected point 1: {reprojected_point_1[:2].ravel()}")
print(f"reprojected point 2: {reprojected_point_2[:2].ravel()}")

error1 = np.linalg.norm(reprojected_point_1[:2].ravel() - undistorted_point_camera_3.ravel())
error2 = np.linalg.norm(reprojected_point_2[:2].ravel() - undistorted_point_camera_4.ravel())
print(f"error 1: {error1}")
print(f"error 2: {error2}")

# TODO: formula for error in meters
cam1_fx = intrinsic_params['CAM_3']['fx']
cam2_fx = intrinsic_params['CAM_4']['fx']

Z = points_3d[2][0]  # depth in mm

error1_m = (error1 * Z) / (cam1_fx * 10) 
error2_m = (error2 * Z) / (cam2_fx * 10) 

print(f"error 1 (cm): {error1_m}")
print(f"error 2 (cm): {error2_m}")

if error1 > threshold or error2 > threshold:
    print("Anomaly detected!")

reprojected point 1: [203.65008615 602.78065921]
reprojected point 2: [3832.98056363  920.31105558]
error 1: 174.47431424269905
error 2: 164.1989594227958
error 1 (cm): 52.30889224329692
error 2 (cm): 55.268097059729655
Anomaly detected!
